# 学习多线程并发
本章学习资料来自https://juejin.im/post/5b17f4305188257d6b5cff6f

In [1]:
import threading



threading.Thread(func, args)来建立一个线程，和多进程差不多

## Lock锁

lock = threading.Lock() 获取锁对象

lock.acquire(); lock.release() 获得、释放锁

也可以用上下文管理器来管理锁，以面忘了释放
```python
with lock:
    pass
```

定义一个简单的任务

In [2]:
def job1():
    global n
    for i in range(5):
        n+=1
        print('job1',n)

def job2():
    global n
    for i in range(5):
        n+=10
        print('job2',n)

n=0
t1=threading.Thread(target=job1)
t2=threading.Thread(target=job2)
t1.start()
t2.start()

job1 1
job1 2
job1 3
job1 4
job1 5
job2 15
job2 25
job2 35
job2 45
job2 55


如果job1对n有要求，不能超过10，那么就会报错。

现在加锁

In [3]:
def job1():
    global n, lock
    with lock:
        for i in range(5):
            n+=1
            print('job1',n)

def job2():
    global n, lock
    with lock:
        for i in range(5):
            n+=10
            print('job2',n)

n = 0
lock = threading.Lock()
t1=threading.Thread(target=job1)
t2=threading.Thread(target=job2)
t1.start()
t2.start()

job1 1
job1 2
job1 3
job1 4
job1 5
job2 15
job2 25
job2 35
job2 45
job2 55


## RLock 可重入锁
当前线程可以反复加锁，但也需要释放同样加锁次数的锁，即重入了多少次，就要释放多少次，不然也会导入锁不被释放

In [5]:
def main():
    n = 0
    # 生成可重入锁对象
    lock = threading.RLock()
    with lock:
        for i in range(10):
            n += 1
            with lock:
                print(n)

t1 = threading.Thread(target=main)
t1.start()

1
2
3
4
5
6
7
8
9
10


## 避免死锁
### 排序
线程A：获取a锁，获取b锁

线程B：获取b锁，获取a锁

==> 可能发生死锁

对a、b锁进行排序，规定必须先拿a再拿b。

缺点：必须知道线程会获得哪些锁
### 超时
超过一定时间则自动释放
### 死锁检测
按照顺序回退进程